In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from collections import Counter

# 1. Przygotowanie bazy (wykonaj raz)
# Używamy wszystkich 9990 wierszy z features_3_sec.csv jako "punktów odniesienia"
reference_data = pd.read_csv("lib/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv")
X_ref = reference_data.drop(columns=['filename', 'length', 'label'])
y_ref = reference_data['label']

# Globalne odchylenie do normalizacji (Z-score)
global_std = X_ref.std()
global_mean = X_ref.mean()

# Normalizujemy całą bazę wzorcową
X_ref_scaled = (X_ref - global_mean) / global_std

def extract_segment_features(y, sr):
    """Ekstrakcja cech z krótkiego fragmentu dźwięku"""
    f = {}
    f['chroma_stft_mean'] = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    f['chroma_stft_var'] = np.var(librosa.feature.chroma_stft(y=y, sr=sr))
    f['rms_mean'] = np.mean(librosa.feature.rms(y=y))
    f['rms_var'] = np.var(librosa.feature.rms(y=y))
    f['spectral_centroid_mean'] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    f['spectral_centroid_var'] = np.var(librosa.feature.spectral_centroid(y=y, sr=sr))
    f['spectral_bandwidth_mean'] = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    f['spectral_bandwidth_var'] = np.var(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    f['rolloff_mean'] = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    f['rolloff_var'] = np.var(librosa.feature.spectral_rolloff(y=y, sr=sr))
    f['zero_crossing_rate_mean'] = np.mean(librosa.feature.zero_crossing_rate(y))
    f['zero_crossing_rate_var'] = np.var(librosa.feature.zero_crossing_rate(y))

    y_harm, y_perc = librosa.effects.hpss(y)
    f['harmony_mean'], f['harmony_var'] = np.mean(y_harm), np.var(y_harm)
    f['perceptr_mean'], f['perceptr_var'] = np.mean(y_perc), np.var(y_perc)

    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    f['tempo'] = tempo[0] if isinstance(tempo, (np.ndarray, list)) else tempo

    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    for i in range(1, 21):
        f[f'mfcc{i}_mean'] = np.mean(mfccs[i-1])
        f[f'mfcc{i}_var'] = np.var(mfccs[i-1])
    return pd.Series(f)


def classify_ultra_precision(file_path, k=7, num_segments=10):
    # Wczytujemy cały plik 30s
    y_full, sr = librosa.load(file_path, duration=30)
    seg_len = 3 * sr

    # Wagi cech - wzmacniamy to, co najlepiej odróżnia muzykę
    weights = pd.Series(1.0, index=X_ref.columns)
    weights[[c for c in X_ref.columns if 'mfcc' in c and 'mean' in c][:5]] *= 2.0 # Barwa
    weights['chroma_stft_mean'] *= 2.5 # Harmonia
    weights['spectral_centroid_mean'] *= 1.5 # Jasność

    segment_votes = []

    for i in range(num_segments):
        start = i * seg_len
        if start + seg_len > len(y_full): break

        y_seg = y_full[start : start + seg_len]

        # Filtrowanie ciszy (puste fragmenty psują wynik)
        if np.max(np.abs(y_seg)) < 0.02: continue

        # Ekstrakcja cech fragmentu i normalizacja
        f_seg = extract_segment_features(y_seg, sr) # funkcja z poprzedniego posta
        f_seg_scaled = (f_seg - global_mean) / global_std

        # Obliczamy odległość Manhattan (często lepsza niż Euklidesowa w audio)
        # sum(|wartość1 - wartość2| * waga)
        diffs = (X_ref_scaled - f_seg_scaled).abs() * weights
        distances = diffs.sum(axis=1)

        # Znajdujemy k najbliższych sąsiadów w całej bazie 10 000 wierszy
        nearest_indices = distances.nsmallest(k).index
        nearest_labels = y_ref.iloc[nearest_indices]

        # Fragment głosuje na najczęstszy gatunek wśród sąsiadów
        segment_votes.append(Counter(nearest_labels).most_common(1)[0][0])

    if not segment_votes: return "unknown"

    # Końcowy werdykt (Majority Voting)
    return Counter(segment_votes).most_common(1)[0][0]

In [2]:
def run_detailed_test(base_dir, limit_per_genre=10):
    results = []
    failed_files = []

    # Pobieramy gatunki na podstawie nazw folderów
    genres = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

    print(f"Rozpoczynam testy (Limit: {limit_per_genre} plików na gatunek)...")

    for real_genre in genres:
        folder_path = os.path.join(base_dir, real_genre)
        # Pobieramy tylko pliki .wav
        files = [f for f in os.listdir(folder_path) if f.endswith('.wav')][:limit_per_genre]

        print(f"--- Testuję: {real_genre.upper()} ---")

        for filename in files:
            file_path = os.path.join(folder_path, filename)

            try:
                # Wywołanie Twojej precyzyjnej funkcji
                predicted_genre = classify_ultra_precision(file_path)

                is_correct = (real_genre == predicted_genre)

                res_entry = {
                    'Plik': filename,
                    'Prawdziwy': real_genre,
                    'Przewidziany': predicted_genre,
                    'Poprawny': is_correct
                }

                results.append(res_entry)

                if not is_correct:
                    failed_files.append(res_entry)
                    print(f"  [X] BŁĄD: {filename} -> Przewidziano: {predicted_genre}")
                else:
                    print(f"  [V] OK: {filename}")

            except Exception as e:
                print(f"  [!] Problem z plikiem {filename}: {e}")

    # Tworzenie raportu końcowego
    df_results = pd.DataFrame(results)
    accuracy = df_results['Poprawny'].mean() * 100

    print("\n" + "="*30)
    print(f"WYNIK KOŃCOWY: {accuracy:.2f}%")
    print("="*30)

    # Wyświetlenie listy błędów dla analizy
    if failed_files:
        print("\nLISTA PLIKÓW, Z KTÓRYMI SOBIE NIE PORADZIŁEM:")
        df_failed = pd.DataFrame(failed_files)
        # Wyświetlamy tylko kolumny Plik, Prawdziwy i Przewidziany
        print(df_failed[['Plik', 'Prawdziwy', 'Przewidziany']].to_string(index=False))

    return df_results

# Uruchomienie:
final_report = run_detailed_test('lib/gtzan-dataset-music-genre-classification/Data/genres_original', limit_per_genre=10)

Rozpoczynam testy (Limit: 10 plików na gatunek)...
--- Testuję: BLUES ---
  [V] OK: blues.00000.wav
  [V] OK: blues.00001.wav
  [V] OK: blues.00002.wav
  [V] OK: blues.00003.wav
  [V] OK: blues.00004.wav
  [V] OK: blues.00005.wav
  [V] OK: blues.00006.wav
  [V] OK: blues.00007.wav
  [V] OK: blues.00008.wav
  [V] OK: blues.00009.wav
--- Testuję: CLASSICAL ---
  [V] OK: classical.00000.wav
  [V] OK: classical.00001.wav
  [V] OK: classical.00002.wav
  [V] OK: classical.00003.wav
  [V] OK: classical.00004.wav
  [V] OK: classical.00005.wav
  [V] OK: classical.00006.wav
  [V] OK: classical.00007.wav
  [V] OK: classical.00008.wav
  [V] OK: classical.00009.wav
--- Testuję: COUNTRY ---
  [V] OK: country.00000.wav
  [V] OK: country.00001.wav
  [V] OK: country.00002.wav
  [V] OK: country.00003.wav
  [V] OK: country.00004.wav
  [V] OK: country.00005.wav
  [V] OK: country.00006.wav
  [V] OK: country.00007.wav
  [V] OK: country.00008.wav
  [V] OK: country.00009.wav
--- Testuję: DISCO ---
  [V] OK: d